In [1]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama import ChatOllama
import pandas as pd

from typing import List
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document

In [20]:
model = "mistral"

llm = ChatOllama(model=model, temparature=0, temperature=0.001)

In [21]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")

In [22]:
llm.with_structured_output(Joke)


RunnableBinding(bound=ChatOllama(model='mistral', temperature=0.001), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Joke', 'description': '', 'parameters': {'type': 'object', 'properties': {'setup': {'description': 'The setup of the joke', 'type': 'string'}, 'punchline': {'description': 'The punchline to the joke', 'type': 'string'}}, 'required': ['setup', 'punchline']}}}], 'tool_choice': 'any'}, config={}, config_factories=[])
| PydanticToolsParser(first_tool_only=True, tools=[<class '__main__.Joke'>])

In [23]:
import pandas as pd
import tiktoken


def num_tokens_from_string(string: str, model: str = "gpt-4o") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(string))
    return num_tokens


news = pd.read_csv(
    "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/news_articles.csv"
)
news["tokens"] = [
    num_tokens_from_string(f"{row['title']} {row['text']}")
    for i, row in news.iterrows()
]
news.head()

title  \
0                             Chevron: Best Of Breed   
1       FirstEnergy (NYSE:FE) Posts Earnings Results   
2  Dáil almost suspended after Sinn Féin TD put p...   
3  Epic’s latest tool can animate hyperrealistic ...   
4  EU to Ban Huawei, ZTE from Internal Commission...   

                                  date  \
0  2031-04-06T01:36:32.000000000+00:00   
1  2030-04-29T06:55:28.000000000+00:00   
2  2023-06-15T14:32:11.000000000+00:00   
3  2023-06-15T14:00:00.000000000+00:00   
4  2023-06-15T13:50:00.000000000+00:00   

                                                text  tokens  
0  JHVEPhoto Like many companies in the O&G secto...      78  
1  FirstEnergy (NYSE:FE – Get Rating) posted its ...     130  
2  The Dáil was almost suspended on Thursday afte...     631  
3  Today, Epic is releasing a new tool designed t...     528  
4  The European Commission is planning to ban equ...     281

In [24]:
from langchain_experimental.graph_transformers import LLMGraphTransformer


llm_transformer = LLMGraphTransformer(
    llm=llm,
    node_properties=["description"],
    relationship_properties=["description"]
)

In [ ]:
from typing import List
from langchain_community.graphs.graph_document import GraphDocument
from langchain_core.documents import Document

def process_text(text: str) -> List[GraphDocument]:
    doc = Document(page_content=text)
    return llm_transformer.convert_to_graph_documents([doc])

In [26]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

MAX_WORKERS = 10
NUM_ARTICLES = 20
graph_documents = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submitting all tasks and creating a list of future objects
    futures = [
        executor.submit(process_text, f"{row['title']} {row['text']}")
        for i, row in news.head(NUM_ARTICLES).iterrows()
    ]

    for future in tqdm(
        as_completed(futures), total=len(futures), desc="Processing documents"
    ):
        graph_document = future.result()
        graph_documents.extend(graph_document)


Processing documents:   0%|          | 0/20 [00:00<?, ?it/s]

Processing documents: 100%|██████████| 20/20 [01:32<00:00,  4.64s/it]


In [27]:
graph_documents

[GraphDocument(nodes=[], relationships=[], source=Document(metadata={}, page_content='XPeng Stock Rises. The Tesla Rival Rolled Out Self-Driving Tech. Chinese electric-vehicle maker\nXPeng\nsaid Thursday its assisted-driving technology has been launched in Beijing and three other cities. The\nTesla\nrival’s stock was rising in premarket trading.')),
 GraphDocument(nodes=[], relationships=[], source=Document(metadata={}, page_content='Ryanair sacks chief pilot over sexual misconduct claims Reuters\nRyanair has sacked its chief pilot after an investigation into his alleged sexual harassment of female colleagues.\nThe airline told staff that he had been fired for "a pattern of repeated inappropriate and unacceptable behaviour towards a number of female pilots".\nThe chief pilot, named in reports as Aidan Murray was appointed in 2020 and had been with the airline for 28 years.\nRyanair declined to comment "on queries relating to individual employees".\nAccording to The Independent, Mr Murr

#### Detemine the number of failed tool calls by the llm

In [28]:
empty_count = 0

for doc in graph_documents:
    if not doc.nodes:
        empty_count += 1

In [29]:
print(f"Percentage missing: {empty_count/len(graph_documents)*100}")

Percentage missing: 75.0
